# Analisando um teste A/B para Farmburg

### Neste projeto analisaremos um teste A/B, com três variantes diferentes (de preços para um pacote de upgrade), para uma empresa que faz um jogo de rede social de simulação agrícola. Usando os módulos do Python, pandas e SciPy, tentaremos chegar a importantes decisões de negócio.

In [1]:
# Importa as bibliotecas

import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import binom_test

In [2]:
# Carrega e analisa os dados

abdata = pd.read_csv('clicks.csv')
abdata.head()

,user_id,group,is_purchase
0,8e27bf9a,A,No
1,eb89e6f0,A,No
2,7119106a,A,No
3,e53781ff,A,No
4,02d48cf1,A,Yes


In [3]:
# Cria tabela de contingencia para o teste Chi-Square

Xtab = pd.crosstab(abdata.group, abdata.is_purchase)
Xtab

is_purchase,No,Yes
group,,
A,1350,316
B,1483,183
C,1583,83


In [4]:
# Calcula o p-value

chi2, pval, dof, expected = chi2_contingency(Xtab)
pval

2.4126213546684264e-35

#### É necessário, de acordo com a demanda da análise, gerar um mínimo de $1000,00 em receita por semana para justificar a execução do projeto de upgrade. Assim analisaremos não apenas o pacote com maior possibilidade de aquisições, mas também aquele que se apresenta mais vantajoso de acordo com a relação ao custo/lucro.

In [5]:
# Calcula o número de visitas a partir dos dados

num_visits = len(abdata)
num_visits

4998

In [19]:
# Calcula o número e a proporção de visitantes que precisariam comprar o pacote a $0.99 para alcançar a receita mínima
num_sales_needed_099 = 1000 / 0.99
p_sales_needed_099 = num_sales_needed_099 / num_visits
num_sales_needed_099, p_sales_needed_099

(1010.1010101010102, 0.20210104243717691)

In [20]:
# Calcula o número e a proporção de visitantes que precisariam comprar o pacote a $1.99 para alcançar a receita mínima
num_sales_needed_199 = 1000 / 1.99
p_sales_needed_199 = num_sales_needed_199 / num_visits
num_sales_needed_199, p_sales_needed_199

(502.51256281407035, 0.10054272965467594)

In [21]:
# Calcula o número e a proporção de visitantes que precisariam comprar o pacote a $4.99 para alcançar a receita mínima
num_sales_needed_499 = 1000 / 4.99
p_sales_needed_499 = num_sales_needed_499 / num_visits
num_sales_needed_499, p_sales_needed_499

(200.40080160320642, 0.040096198800161346)

In [22]:
# Analisa os clientes do grupo A ($ 0.99) e aqueles desse grupo de efetivaram a compra

samp_size_099 = np.sum(abdata.group == 'A')
sales_099 = np.sum((abdata.group == 'A') & (abdata.is_purchase == 'Yes'))
samp_size_099, sales_099

(1666, 316)

In [23]:
# Analisa os clientes do grupo B ($ 1.99) e aqueles desse grupo de efetivaram a compra

samp_size_199 = np.sum(abdata.group == 'B')
sales_199 = np.sum((abdata.group == 'B') & (abdata.is_purchase == 'Yes'))
samp_size_199, sales_199

(1666, 183)

In [26]:
# Analisa os clientes do grupo C ($ 4.99) e aqueles desse grupo de efetivaram a compra

samp_size_499 = np.sum(abdata.group == 'C')
sales_499 = np.sum((abdata.group == 'C') & (abdata.is_purchase == 'Yes'))
samp_size_499, sales_499

(1666, 83)

In [27]:
# Realiza um teste binomial com a taxa de compra observada e a necessária para o grupo A

pvalueA = binom_test(x=sales_099, n=samp_size_099, p=p_sales_needed_099, alternative='greater')
pvalueA

0.9028081076188554

In [28]:
# Realiza um teste binomial com a taxa de compra observada e a necessária para o grupo B

pvalueB = binom_test(x=sales_199, n=samp_size_199, p=p_sales_needed_199, alternative='greater')
pvalueB

0.11184562623740614

In [29]:
# Realiza um teste binomial com a taxa de compra observada e a necessária para o grupo C

pvalueC = binom_test(x=sales_499, n=samp_size_499, p=p_sales_needed_499, alternative='greater')
pvalueC

0.02794482665983064

#### Percebemos que pvalueC é o único valor p abaixo do limite de 0,05. Portanto, o grupo C é o único grupo em que a taxa de compra é significativamente maior do que a meta necessária para atingir uma receita de \$ 1000 por semana. Portanto, o stakeholder deveria cobrar o valor de \\$ 4,99 pela atualização.